In [13]:
import json
import torch, torch.nn as nn
from torch.autograd import Variable
import random


with open("/Users/clarkwang/Documents/GitHub/LING_111/Project/preprocessed_data/bnc_spoken_output_flattened.json") as infile:
    input_data = json.load(infile)
    text = input_data[0]

# print(text)


seq_len = len(text)
batch_size = 1
embedding_size = 1
hidden_size = 1

# get max sequence length, then pad every sentence

vocab = set()

for sentence in input_data[0:10]:
    vocab = vocab.union(sentence)

vocab = list(vocab)
vocab_size = len(vocab)
output_size = vocab_size
word_ids = {word: id_ for id_, word in enumerate(vocab)}
ids_word = {id_: word for word, id_ in word_ids.items()}

# text = input_data[random.randint(0, vocab_size - 1)]
indices = [word_ids[word] for word in text]
input_val = torch.eye(vocab_size)[indices].unsqueeze(1)

bi_rnn = torch.nn.RNN(
    input_size=vocab_size, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)

bi_output, bi_hidden = bi_rnn(input_val)

# stagger
forward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]
staggered_output = torch.cat((forward_output, backward_output), dim=-1)

linear = nn.Linear(hidden_size * 2, output_size)

# only predict on words
labels = torch.LongTensor(indices[1:-1])

# for language models, use cross-entropy :)
loss = nn.CrossEntropyLoss()
output = loss(linear(staggered_output.squeeze(1)), labels)

print(output)

optimizer = optim.Adam(rnn_classifier_newstart_adam.parameters())

'''
for epoch in range(epochs):
    # Within each epoch, iterate over the data in mini-batches
    # Note the use of *datapoint_list for generality, whether or not there are offsets
    for (label_list, *datapoint_list) in dataloader:
            
        # Clear out gradients accumulated from inputs in the previous mini-batch
        model.zero_grad()

        # Run the forward pass to make predictions for the mini-batch
        predicted_probs = model(*datapoint_list).view(-1)

        # Compute the loss and send it backward through the network to get gradients
        # Note: PyTorch averages the loss over all datapoints in the minibatch
        loss = model.loss_function(predicted_probs, label_list.to(torch.float32))
        loss.backward()
            
        # Nudge the weights
        optimizer.step()
            
        # Track performance
        if print_every is not None: 
            total_acc += ((predicted_probs > 0.5).to(torch.int64) == label_list).sum().item()
            total_count += label_list.size(0)
            current_loss += loss.item()

        # Log performance
        if print_every is not None and (epoch + 1) % print_every == 0:
            log_message = ('| epoch {:3d} | train loss {:6.3f} |'
                           .format(epoch + 1, total_acc/total_count, current_loss/minibatches_per_log))
            if validation_data is not None:
                validation_performance = test(model, validation_data)
                log_message += ' valid loss {loss:6.3f} |'.format(**validation_performance)
            print(log_message)

            # Reset trackers after logging
            total_acc = 0
            total_count = 0
            current_loss = 0.0
            model.train()
'''

tensor(5.5016, grad_fn=<NllLossBackward0>)


In [48]:
class biRNNLM(nn.Module):

    def __init__(self, vocab, hidden_size, freeze_embeddings=True,
                 recurrent_activation="tanh", recurrent_layers=1, recurrent_bidirectional=False):
        super(biRNNLM, self).__init__()
        
        self.vocab = vocab
        
        self.embedding = nn.Embedding(vocab_size, 10)
        
        # The embeddings go into an RNN layer with recurrent_dim units
        self.bi_rnn = torch.nn.RNN(input_size=vocab_size, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)

        self.linear = nn.Linear(hidden_size * 2, output_size)
        
        self.loss_function = nn.CrossEntropyLoss()
        
    def forward(self, text, seq_lengths):
        
        indices = [word_ids[word] for word in text]
        input_val = torch.eye(vocab_size)[indices].unsqueeze(1)
        
        word_embeddings = self.embedding(input_val)
        
        bi_output, bi_hidden = self.bi_rnn(word_embeddings)

        # stagger
        forward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]
        staggered_output = torch.cat((forward_output, backward_output), dim=-1)

        # only predict on words
        labels = torch.LongTensor(indices[1:-1])

        # for language models, use cross-entropy :)

        output = self.loss_function(self.linear(staggered_output.squeeze(1)), labels)
        
        return output
        

KeyError: 'does'

In [ ]:


optimizer = optim.Adam(

for epoch in range(epochs):
    # Within each epoch, iterate over the data in mini-batches
    # Note the use of *datapoint_list for generality, whether or not there are offsets
    for (label_list, *datapoint_list) in dataloader:
            
        # Clear out gradients accumulated from inputs in the previous mini-batch
        model.zero_grad()

        # Run the forward pass to make predictions for the mini-batch
        predicted_probs = model(*datapoint_list).view(-1)

        # Compute the loss and send it backward through the network to get gradients
        # Note: PyTorch averages the loss over all datapoints in the minibatch
        loss = model.loss_function(predicted_probs, label_list.to(torch.float32))
        loss.backward()
            
        # Nudge the weights
        optimizer.step()
            
        # Track performance
        if print_every is not None: 
            total_acc += ((predicted_probs > 0.5).to(torch.int64) == label_list).sum().item()
            total_count += label_list.size(0)
            current_loss += loss.item()

        # Log performance
        if print_every is not None and (epoch + 1) % print_every == 0:
            log_message = ('| epoch {:3d} | train loss {:6.3f} |'
                           .format(epoch + 1, total_acc/total_count, current_loss/minibatches_per_log))
            if validation_data is not None:
                validation_performance = test(model, validation_data)
                log_message += ' valid loss {loss:6.3f} |'.format(**validation_performance)
            print(log_message)

            # Reset trackers after logging
            total_acc = 0
            total_count = 0
            current_loss = 0.0
            model.train()